In [1]:
import os
import random
import sys
from pathlib import Path

# Set the path to the directory containing the project
sys.path.append('D:\\Internship\\recsys\\data_pipelines')
sys.path.append('D:\\Internship\\recsys\\data_pipelines\\dags')
sys.path.append('D:\\Internship\\recsys\\movie_recommendation_system\\src')
sys.path.append('D:\\Internship\\recsys\\db_handlers')
sys.path.append('D:\\Internship\\recsys\\data')

In [2]:
from db_handlers.user_postgres_sql_db_handler import store_new_user
from db_handlers.user_mongodb_nosql_db_handler import store_user_movie_rating
from movie_recommender.data.tabular_dataset_handler import TabularDatasetHandler
from offline_pipeline import (
    load_last_ratings,
    load_last_offline_gnn_model,
    offline_train_gnn_model,
    deploy_model,
)

c:\Users\Public\anaconda3\envs\tf-gpu\Lib\site-packages\airflow\__init__.py:36: RuntimeWarning: Airflow currently can be run on POSIX-compliant Operating Systems. For development, it is regularly tested on fairly modern Linux Distros and recent versions of macOS. On Windows you can run it via WSL2 (Windows Subsystem for Linux 2) or via Linux Containers. The work to add Windows support is tracked via https://github.com/apache/airflow/issues/10388, but it is not a high priority.
  warnings.warn(
OSError while attempting to symlink the latest log directory


[LOG] Detected local environment, with Docker: [False]
[LOG] Detected local environment, with Docker: [False]
[INFO] Running locally
[LOG] Detected local environment, with Docker: [False]
[LOG] Detected local environment, with Docker: [False]


In [3]:
import uuid

# Create a new test user
test_user_id = "bdd62e39-8999-468b-be8a-c36277a93bdc" #str(uuid.uuid4())
user_name = "Christian Francesco"
user_surname = "Russo"
user_email = "rus.chr.20499@gmail.com"

# Store the new user in the PostgreSQL database
store_new_user(test_user_id, user_name, user_surname, user_email) 

[LOG] Assigned new model ID 1002 to user bdd62e39-8999-468b-be8a-c36277a93bdc in 'user_model_lookup' table.
[LOG] New user bdd62e39-8999-468b-be8a-c36277a93bdc stored successfully in 'user_register' table


In [4]:
# Load movies DataFrame
tdh_filepath = "D:\\Internship\\recsys\\data\\movielens_processed\\tdh_instance.pkl"
tdh = TabularDatasetHandler.load_class_instance(tdh_filepath)
movies_df = tdh.get_movies_df_deepcopy()

# Sample 5 movies and store synthetic new ratings
sampled_movies = movies_df.sample(5)
for _, row in sampled_movies.iterrows():
    rating_value = round(random.uniform(1.0, 5.0), 1)
    print(f"Movie '{row['title']}' => rating {rating_value}")
    store_user_movie_rating(
        user_id=test_user_id,
        movie_id=int(row['id']),
        rating=rating_value
    )

Movie 'For Elise' => rating 4.9
[LOG] New user rating stored successfully in 'user_movie_ratings' collection.
Movie 'Cottage to Let' => rating 3.8
[LOG] New user rating stored successfully in 'user_movie_ratings' collection.
Movie 'Lenny Bruce: Swear to Tell the Truth' => rating 3.3
[LOG] New user rating stored successfully in 'user_movie_ratings' collection.
Movie 'Valley Girl' => rating 2.2
[LOG] New user rating stored successfully in 'user_movie_ratings' collection.
Movie 'The Adventures of Sherlock Holmes and Doctor Watson: King of Blackmailers' => rating 4.1
[LOG] New user rating stored successfully in 'user_movie_ratings' collection.


In [5]:
test_user_id = "bdd62e39-8999-468b-be8a-c36277a93bdc"
test_num_new_ratings = 5
print(f"Testing user {test_user_id} with {test_num_new_ratings} new ratings.")

Testing user bdd62e39-8999-468b-be8a-c36277a93bdc with 5 new ratings.


In [6]:
load_last_ratings()

[LOG] Loading the last user-movie ratings


'load_last_offline_gnn_model'

In [7]:
load_last_offline_gnn_model()

[LOG] Loading the last offline GNN model
[LOG] Latest offline updated model '2025-04-08_offline_model.pth' downloaded as 'old_GNN_model.pth'.


In [8]:
offline_train_gnn_model()

[LOG] Offline training GNN model with new user-movie ratings
[INFO] Running CUDA script [train_offline_gnn_script.py] as a subprocess
[ERROR] Failed to execute script as subprocess: D:\Internship\recsys\data_pipelines\scripts\offline\train_offline_gnn_script.py
Traceback (most recent call last):
  File "D:\Internship\recsys\data_pipelines\scripts\offline\train_offline_gnn_script.py", line 88, in <module>
    main()
  File "D:\Internship\recsys\data_pipelines\scripts\offline\train_offline_gnn_script.py", line 79, in main
    GraphSAGE_model.incremental_train(
  File "D:\Internship\recsys\movie_recommendation_system\src\movie_recommender\models\gnn_retrain_strategies.py", line 297, in incremental_train
    raise ValueError(
ValueError: Impossible to re-train the model: no new edges to add to the training set. Current subgraph structure: HeteroData(
  movie={
    node_id=[45433],
    x=[45433, 405],
  },
  (user, rating, movie)={}
)



In [9]:
deploy_model()

[LOG] Deploying the updated offline GNN model
[LOG] Offline updated model '2025-04-08_offline_model.pth' uploaded.
